In [1]:
import os
path_to_spark = "/usr/hdp/2.3.4.0-3485/spark/"
os.environ['SPARK_HOME'] = path_to_spark
import sys
path_to_pyspark = os.path.join(path_to_spark, "python")
sys.path.insert(0, path_to_pyspark)
path_to_py4j ="/usr/hdp/2.3.4.0-3485/spark/python/lib/py4j-0.8.2.1-src.zip"
sys.path.insert(0, path_to_py4j)
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS 
import math
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
import pyspark.sql.functions as func
from math import sqrt

In [2]:
sc = SparkContext()

In [3]:
nba_data = sc.textFile("/nba/nba_all.csv")

In [4]:
nba_data_header = nba_data.take(1)[0]

In [5]:
nba_data = nba_data.filter(lambda line: line!=nba_data_header).map(lambda line: line.split(","))

In [6]:
nba_data = nba_data.collect()

In [7]:
def nba_mean(dim) :
    Dimension = {'MIN':1,'PTS':2,'AST':3,'TOV':4,'eFG':5,'TS':6}
    a = []
    for x in nba_data :
        a.append(float(x[Dimension[dim]]))
    data_mean = sc.parallelize(a).mean()
    return data_mean

def nba_std(dim) :
    Dimension = {'MIN':1,'PTS':2,'AST':3,'TOV':4,'eFG':5,'TS':6}
    a = []
    for x in nba_data :
        a.append(float(x[Dimension[dim]]))
    data_std = sc.parallelize(a).stdev()
    return data_std
  
MIN_data_mean = nba_mean('MIN')
MIN_data_std = nba_std('MIN')
PTS_data_mean = nba_mean('PTS')
PTS_data_std = nba_std('PTS')
AST_data_mean = nba_mean('AST')
AST_data_std = nba_std('AST')
TOV_data_mean = nba_mean('TOV')
TOV_data_std = nba_std('TOV')
eFG_data_mean = nba_mean('eFG')
eFG_data_std = nba_std('eFG')
TS_data_mean = nba_mean('TS')
TS_data_std = nba_std('TS')
D1 = []

for x in nba_data:
    MIN = (((float(x[1])- MIN_data_mean) / MIN_data_std  ) - ((float(nba_data[0][1])- MIN_data_mean) / MIN_data_std  ))**2
    PTS = (((float(x[2])- PTS_data_mean) / PTS_data_std  ) - ((float(nba_data[0][2])- PTS_data_mean) / PTS_data_std  ))**2
    AST = (((float(x[3])- AST_data_mean) / AST_data_std  ) - ((float(nba_data[0][3])- AST_data_mean) / AST_data_std  ))**2
    TOV = (((float(x[4])- TOV_data_mean) / TOV_data_std  ) - ((float(nba_data[0][4])- TOV_data_mean) / TOV_data_std  ))**2
    eFG = (((float(x[5])- eFG_data_mean) / eFG_data_std  ) - ((float(nba_data[0][5])- eFG_data_mean) / eFG_data_std  ))**2
    TS =  (((float(x[6])- TS_data_mean) / TS_data_std  ) - ((float(nba_data[0][6])- TS_data_mean) / TS_data_std  ))**2
    distance = sqrt(MIN + PTS + AST + TOV + eFG + TS)
    name = str(x[0])
    c = name + ',' + str(distance)
    D1.append(c)




    
    


    
#for x in nba_data:
#    MIN = (float(x[1]) - float(nba_data[0][1]))**2
#    PTS = (float(x[2]) - float(nba_data[0][2]))**2
#    AST = (float(x[3]) - float(nba_data[0][3]))**2
#    TOV = (float(x[4]) - float(nba_data[0][4]))**2
#    eFG = (float(x[5]) - float(nba_data[0][5]))**2
#    TS =  (float(x[6]) - float(nba_data[0][6]))**2
#    distance = sqrt(MIN + PTS + AST + TOV + eFG + TS)
#    name = str(x[0])
#    print str(x[0]) +  "," + str(distance)
    

In [8]:
Rank = sc.parallelize(D1)

In [9]:
splits = Rank.map(lambda x: x.split(','))

In [10]:
sqlContext = SQLContext(sc)

In [11]:
Ranks = splits.map(lambda p: Row(name=p[0], distance=float(p[1])))

In [12]:
schemaPeople = sqlContext.createDataFrame(Ranks)

In [13]:
schemaPeople.show()

+-------------+-------------------+
|     distance|               name|
+-------------+-------------------+
|          0.0|     Michael Jordan|
|2.08500025176|       Kevin Durant|
|2.03610771872|       LeBron James|
|3.46053034079|      Allen Iverson|
|2.58912602339|      George Gervin|
|2.18681102208|        Karl Malone|
|2.33184743121|        Kobe Bryant|
|2.56676507767|    Carmelo Anthony|
|3.00153365185|  Dominique Wilkins|
|2.90037324492|Kareem Abdul-Jabbar|
| 2.2199797812|         Larry Bird|
|3.17803345479|     Adrian Dantley|
|4.29630720065|      Pete Maravich|
|3.95295470197|   Shaquille O'Neal|
|2.75234343481|        Dwyane Wade|
|3.78579454425|         Rick Barry|
|3.42260565446|       Bernard King|
| 4.1996135256|      Stephen Curry|
|3.75655973779|    Charles Barkley|
|3.73359450012|     David Thompson|
+-------------+-------------------+
only showing top 20 rows



In [14]:
schemaPeople.sort("distance",ascending=True).show()

+-------------+-------------------+
|     distance|               name|
+-------------+-------------------+
|          0.0|     Michael Jordan|
|2.03610771872|       LeBron James|
|2.08500025176|       Kevin Durant|
|2.18681102208|        Karl Malone|
| 2.2199797812|         Larry Bird|
|2.33184743121|        Kobe Bryant|
|2.56676507767|    Carmelo Anthony|
|2.58912602339|      George Gervin|
|2.75234343481|        Dwyane Wade|
|2.90037324492|Kareem Abdul-Jabbar|
|3.00153365185|  Dominique Wilkins|
|3.17803345479|     Adrian Dantley|
| 3.3245867956|      Blake Griffin|
|3.33655481641|      Julius Erving|
|3.42260565446|       Bernard King|
|3.45578126491|    Hakeem Olajuwon|
|3.46053034079|      Allen Iverson|
|3.53826669164|     Mitch Richmond|
|  3.724616571|         Bob McAdoo|
|3.73287577593|      Clyde Drexler|
+-------------+-------------------+
only showing top 20 rows

